In [28]:
import pandas as pd
import ssl
import numpy as np
from pathlib import Path
import sqlalchemy
import datetime

# from utils.helpers import alpaca_symbol_data

# For creating the DB
database_connection_string = 'sqlite:///SP500.db'
engine = sqlalchemy.create_engine(database_connection_string)

# prevent an ssl failure
ssl._create_default_https_context = ssl._create_unverified_context

payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

# There are 2 tables on the Wikipedia page
# we want the first table
first_table = payload[0]
second_table = payload[1]

df500table = first_table

sp500Tickers = df500table['Symbol'].values.tolist()
df500sectors = df500table.drop(columns={'Security','SEC filings','GICS Sub-Industry', 'Headquarters Location', 'Date first added', 'CIK','Founded'})
df500sectors.set_index('Symbol', inplace=True)
sectors = df500sectors['GICS Sector'].unique()
# print(sp500Tickers)

In [29]:
StocksWide = pd.read_sql_table('StocksWide', engine)
StocksWide.drop(columns='index', inplace=True)
StocksWide = StocksWide.set_index('timestamp')
StocksWide.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS,SPY Open,SPY Close
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-04-13 04:00:00,0.010410,0.016216,-0.007205,0.015125,0.001759,0.001968,-0.011522,0.016192,-0.004815,0.015665,...,0.008143,0.002355,-0.010348,0.008414,0.005851,-0.010332,-0.007684,0.007830,411.57,412.86
2021-04-14 04:00:00,-0.008490,-0.025272,0.001600,-0.022000,-0.003013,0.009064,0.002824,-0.006947,0.000844,-0.006556,...,0.017936,0.018744,0.007477,0.005360,-0.003552,-0.005917,0.015529,-0.005670,412.86,411.45
2021-04-15 04:00:00,0.004598,-0.021653,0.006214,0.004781,0.008491,0.005066,0.012594,0.013658,-0.003878,0.014951,...,-0.004716,-0.005371,0.001019,-0.007259,0.018743,0.005353,-0.011160,0.009272,413.79,415.87
2021-04-16 04:00:00,0.003954,-0.003169,0.008574,-0.003308,0.003907,0.001590,-0.026277,-0.000322,0.006312,0.002310,...,-0.010651,-0.000306,-0.005585,-0.004889,-0.004979,0.004952,-0.004079,0.008179,417.29,417.26
2021-04-19 04:00:00,-0.006549,-0.019563,-0.005424,0.010110,0.005928,-0.004167,0.003415,0.007524,-0.001951,-0.013419,...,-0.005634,0.000611,-0.006974,-0.005660,0.003548,-0.007537,-0.004802,0.003582,416.26,415.21


In [30]:
# StocksWide['SPY Returns'] = 0.0
# StocksWide.loc[(StocksWide['SPY'] >= 0), 'SPY Returns'] = 1
# StocksWide.loc[(StocksWide['SPY'] < 0), 'SPY Returns'] = 0
# StocksWide.drop(columns='SPY', inplace=True)
# StocksWide['SPY Returns'] = StocksWide['SPY Returns'].shift(-1)
# StocksWide.rename(columns={'SPY Returns':'SPY'}, inplace=True)
# StocksWide.dropna(inplace=True)

SectorDF = StocksWide[['SPY', 'SPY Open', 'SPY Close']]
SectorDF

,SPY,SPY Open,SPY Close
timestamp,,,
2021-04-13 04:00:00,0.003134,411.57,412.86
2021-04-14 04:00:00,-0.003415,412.86,411.45
2021-04-15 04:00:00,0.005027,413.79,415.87
2021-04-16 04:00:00,-0.000072,417.29,417.26
2021-04-19 04:00:00,-0.002522,416.26,415.21
...,...,...,...
2022-04-07 04:00:00,0.006956,445.67,448.77
2022-04-08 04:00:00,-0.001049,448.04,447.57
2022-04-11 04:00:00,-0.009435,444.11,439.92


In [31]:
# Iterates through StocksWide and puts all the stocks into a sector before taking a mean

stockstodrop = []
StocksWideNoSPY = StocksWide.drop(columns={'SPY','SPY Open','SPY Close'})
for sector in sectors:
    for (columnName, columnValues) in StocksWideNoSPY.iteritems():
        if sector != df500sectors.loc[columnName]['GICS Sector']:
            stockstodrop.append(columnName)
    TempDF = StocksWideNoSPY.drop(columns=stockstodrop)
    stockstodrop = []
    SectorDF[sector] = TempDF.mean(axis=1)


C:\Users\canav\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [32]:
SectorDF.head()

,SPY,SPY Open,SPY Close,Industrials,Health Care,Information Technology,Communication Services,Consumer Staples,Consumer Discretionary,Utilities,Financials,Materials,Real Estate,Energy
timestamp,,,,,,,,,,,,,,
2021-04-13 04:00:00,0.003134,411.57,412.86,-0.001641,0.006510,0.000045,-0.000767,0.000146,-0.000002,0.016096,-0.004232,-0.003210,0.007684,0.001140
2021-04-14 04:00:00,-0.003415,412.86,411.45,-0.000804,0.000509,-0.004617,-0.004491,-0.001597,-0.002539,0.007733,0.010694,0.009282,-0.004979,0.020342
2021-04-15 04:00:00,0.005027,413.79,415.87,0.000453,0.011926,0.004855,0.003417,0.005316,-0.004535,0.011355,-0.001082,0.004324,0.011913,-0.006888
2021-04-16 04:00:00,-0.000072,417.29,417.26,-0.003498,0.002819,-0.002692,-0.003881,0.001754,0.002421,0.002735,-0.004246,-0.001499,-0.003487,-0.018957
2021-04-19 04:00:00,-0.002522,416.26,415.21,-0.004149,-0.000356,-0.009452,-0.002727,-0.002429,-0.007847,-0.005995,-0.003404,-0.005562,0.002253,-0.001944


In [33]:
SectorDF['SPY Returns'] = 0.0
SectorDF.loc[(SectorDF['SPY'] >= 0), 'SPY Returns'] = 1
SectorDF.loc[(SectorDF['SPY'] < 0), 'SPY Returns'] = 0
SectorDF.drop(columns='SPY', inplace=True)
SectorDF['SPY Returns'] = SectorDF['SPY Returns'].shift(-1)
SectorDF['SPY Open'] = SectorDF['SPY Open'].shift(-1)
SectorDF['SPY Close'] = SectorDF['SPY Close'].shift(-1)
SectorDF.rename(columns={'SPY Returns':'SPY'}, inplace=True)
SectorDF.dropna(inplace=True)
SectorDF.head()

C:\Users\canav\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\canav\anaconda3\envs\dev\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\canav\anaconda3\envs\dev\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

,SPY Open,SPY Close,Industrials,Health Care,Information Technology,Communication Services,Consumer Staples,Consumer Discretionary,Utilities,Financials,Materials,Real Estate,Energy,SPY
timestamp,,,,,,,,,,,,,,
2021-04-13 04:00:00,412.86,411.45,-0.001641,0.006510,0.000045,-0.000767,0.000146,-0.000002,0.016096,-0.004232,-0.003210,0.007684,0.001140,0.0
2021-04-14 04:00:00,413.79,415.87,-0.000804,0.000509,-0.004617,-0.004491,-0.001597,-0.002539,0.007733,0.010694,0.009282,-0.004979,0.020342,1.0
2021-04-15 04:00:00,417.29,417.26,0.000453,0.011926,0.004855,0.003417,0.005316,-0.004535,0.011355,-0.001082,0.004324,0.011913,-0.006888,0.0
2021-04-16 04:00:00,416.26,415.21,-0.003498,0.002819,-0.002692,-0.003881,0.001754,0.002421,0.002735,-0.004246,-0.001499,-0.003487,-0.018957,0.0
2021-04-19 04:00:00,413.93,412.17,-0.004149,-0.000356,-0.009452,-0.002727,-0.002429,-0.007847,-0.005995,-0.003404,-0.005562,0.002253,-0.001944,0.0


In [86]:
(SectorDF+1).cumprod()

,Industrials,Health Care,Information Technology,Communication Services,Consumer Staples,Consumer Discretionary,Utilities,Financials,Materials,Real Estate,Energy,SPY
timestamp,,,,,,,,,,,,
2021-04-13 04:00:00,0.998359,1.006510,1.000045,0.999233,1.000146,0.999998,1.016096,0.995768,0.996790,1.007684,1.001140,1.000000e+00
2021-04-14 04:00:00,0.997556,1.007023,0.995428,0.994745,0.998548,0.997459,1.023953,1.006417,1.006042,1.002667,1.021505,2.000000e+00
2021-04-15 04:00:00,0.998008,1.019033,1.000262,0.998143,1.003856,0.992936,1.035580,1.005328,1.010392,1.014612,1.014469,2.000000e+00
2021-04-16 04:00:00,0.994517,1.021905,0.997569,0.994270,1.005617,0.995339,1.038413,1.001059,1.008877,1.011074,0.995238,2.000000e+00
2021-04-19 04:00:00,0.990391,1.021541,0.988140,0.991558,1.003175,0.987529,1.032188,0.997652,1.003266,1.013351,0.993304,2.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-06 04:00:00,0.881327,1.121450,0.956211,0.863526,1.042492,0.831203,1.113455,0.922775,0.990149,1.075692,1.333758,6.805647e+38
2022-04-07 04:00:00,0.886488,1.145055,0.961191,0.855463,1.051411,0.835105,1.109873,0.921410,0.994521,1.070898,1.337260,6.805647e+38
2022-04-08 04:00:00,0.881354,1.151267,0.954327,0.857057,1.053398,0.838848,1.111336,0.924081,0.996530,1.072714,1.374671,6.805647e+38


In [34]:
SectorDF.to_sql('SectorDF', engine, if_exists='replace')

# Market Cap Sleeves

Work in process, do not reference past this

In [35]:
SP500_Constituents_details = pd.read_csv(Path("resources/constituents_financials.csv"), index_col='Symbol')
SP500_Constituents_details = SP500_Constituents_details.sort_values(by=['Market Cap'], ascending=False)
SP500_Constituents_details.head()

,Name,Sector,Price,Price/Earnings,Dividend Yield,Earnings/Share,52 Week Low,52 Week High,Market Cap,EBITDA,Price/Sales,Price/Book,SEC Filings
Symbol,,,,,,,,,,,,,
AAPL,Apple Inc.,Information Technology,155.15,16.86,1.579541,9.20,180.10,131.1200,8.095080e+11,7.938600e+10,3.458609,5.66,http://www.sec.gov/cgi-bin/browse-edgar?action...
GOOGL,Alphabet Inc Class A,Information Technology,1007.71,31.48,0.000000,22.27,1198.00,824.3000,7.338240e+11,3.421700e+10,6.801692,4.70,http://www.sec.gov/cgi-bin/browse-edgar?action...
GOOG,Alphabet Inc Class C,Information Technology,1001.52,40.29,0.000000,22.27,1186.89,803.1903,7.285360e+11,3.271400e+10,6.772653,4.67,http://www.sec.gov/cgi-bin/browse-edgar?action...
MSFT,Microsoft Corp.,Information Technology,85.01,25.76,1.874791,2.97,96.07,63.2200,6.899780e+11,4.107900e+10,7.113097,9.49,http://www.sec.gov/cgi-bin/browse-edgar?action...
AMZN,Amazon.com Inc,Consumer Discretionary,1350.50,296.16,0.000000,6.16,1498.00,812.5000,6.858730e+11,1.613200e+10,3.927053,24.28,http://www.sec.gov/cgi-bin/browse-edgar?action...


In [13]:
StockCount = 0
Sleeve = 1
StockMarketCap = [[],[],[],[],[],[]]

for index, row in SP500_Constituents_details.iterrows():
    if StockCount == Sleeve * 100:
        Sleeve += 1
    StockMarketCap[Sleeve-1].append(index)
    StockCount += 1

In [14]:
StocksWide.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-04-13 04:00:00,0.010410,0.016216,-0.007205,0.015125,0.001759,0.001968,-0.011522,0.016192,-0.004815,0.015665,...,0.011212,0.013618,0.008143,0.002355,-0.010348,0.008414,0.005851,-0.010332,-0.007684,0.007830
2021-04-14 04:00:00,-0.008490,-0.025272,0.001600,-0.022000,-0.003013,0.009064,0.002824,-0.006947,0.000844,-0.006556,...,-0.007537,0.004826,0.017936,0.018744,0.007477,0.005360,-0.003552,-0.005917,0.015529,-0.005670
2021-04-15 04:00:00,0.004598,-0.021653,0.006214,0.004781,0.008491,0.005066,0.012594,0.013658,-0.003878,0.014951,...,-0.014030,0.020566,-0.004716,-0.005371,0.001019,-0.007259,0.018743,0.005353,-0.011160,0.009272
2021-04-16 04:00:00,0.003954,-0.003169,0.008574,-0.003308,0.003907,0.001590,-0.026277,-0.000322,0.006312,0.002310,...,-0.005785,0.001423,-0.010651,-0.000306,-0.005585,-0.004889,-0.004979,0.004952,-0.004079,0.008179
2021-04-19 04:00:00,-0.006549,-0.019563,-0.005424,0.010110,0.005928,-0.004167,0.003415,0.007524,-0.001951,-0.013419,...,-0.002561,-0.000567,-0.005634,0.000611,-0.006974,-0.005660,0.003548,-0.007537,-0.004802,0.003582


In [15]:
SleeveDf = StocksWide[['SPY']]
SleeveDf

,SPY
timestamp,
2021-04-13 04:00:00,0.003134
2021-04-14 04:00:00,-0.003415
2021-04-15 04:00:00,0.005027
2021-04-16 04:00:00,-0.000072
2021-04-19 04:00:00,-0.002522
...,...
2022-04-07 04:00:00,0.006956
2022-04-08 04:00:00,-0.001049
2022-04-11 04:00:00,-0.009435


In [16]:
SleeveDf = StocksWide[['SPY']]

for sleeve in StockMarketCap:
    for stock in sleeve:
        TempDF
        